# Objectif:

On souhaite évaluer le nombre de publications qui sont le résultat d'études liées à l'INSERM.

En particulier, on souhaite identifier les études qui ne donnent lieu à aucune publication et essayer de comprendre les raisons de cette absence de publication.

# Organisation :
Dans ce premier notebook, l'objectif est d'extraire de manière automatique les données des études:

- On récupère via l'API de *ClinicalTrial* les *ID* (**NCTId**) des études qui ont :
  + pour sponsors l'*INSERM*, l'*ANRS* etc.
  + un statut 'COMPLETED'
  + une date de fin d'étude postérieure à 2013

- A partir de ces **NCTIds**:
   + on récupère sur *ClinicalTrial* les informations liées à ces études, en particulier des publications et leurs **PMIDs**.  
    Ces publications sont de 2 types:
        1. Elles ont été uploadées sur *CT* par les auteurs de l'étude: `BACKGROUND, RESULT`
        2. Elles ont été automatiquement récupérés sur PubMed par *CT*: `DERIVED`
    + on récupère sur *PubMed* les **PMIDs** des publications liées à ces études.  
      On retrouve général un peu plus de publications que le traitement automatique réalisé par *CT*. 

- Pour chaque **NCTIds**, on fusionne l'ensemble des **PMIDs** retournées par *CT* et *PubMed*.


- A partir de cet ensemble de **PMIDs**, on récupère les infos des publications via *Pubmed*: `titre, auteurs, doi`...

- On récupère le status OA avec le DOI et l'API d'Unpaywall

- On sauvegarde le résultat sous la forme d'une fichier CSV.

# Extraction des NCTIds dans ClinicalTrial:

<span style="color:red">**Obsolète**</span>  
<span style="color:red">**L'API v1 n'est plus disponible depuis mi-2024.**  
**Aller directement à [la section API v2](#API-v2:)**</span>

## API v1:

Pour faciliter la récupération des données via l'API v1 de ClinicalTrial, on utilise le wrapper Python: [pytrials](https://github.com/jvfe/pytrials)

Installer ***pytrials***:
- Dans Conda Navigator, se placer dans le meme environnement que celui qui execute Jupyter
- Lancer Powershell Prompt dans cet environnement
- Taper: `pip install pytrials`

In [1]:
# from pytrials.client import ClinicalTrials
# import urllib.parse

In [2]:
# ct = ClinicalTrials()

### Création de la requête:

On crée la requête qui sera envoyé à l'API de ClinicalTrial

#### Sponsors:

In [3]:
# sponsors = [
#     'anrs',
#     'inserm',
#     'institut national de la santé et de la recherche médicale',
#     'french national agency for research on aids and viral hepatitis',
# ]

In [4]:
# sponsors_expr = [f'AREA[LeadSponsorName]{sponsor}' for sponsor in sponsors]

# # Add OR keyword
# sponsors_expr = ' OR '.join(sponsors_expr)

# # Add parenthesis for correct interpretation of OR expression
# sponsors_expr = f'({sponsors_expr})'

# sponsors_expr

#### Status:

In [5]:
# status = 'completed'

In [6]:
# status_expr = f'AREA[OverallStatus]{status}'
# status_expr

#### Date de fin d'étude supérieure ou égale à 2013:

<span style="color:red">**Mettre à jour la date si nécessaire**</span>

In [7]:
# date_expr = 'AREA[CompletionDate]RANGE[01/01/2013,MAX]'
# date_expr

#### Search Expression:

In [8]:
# search_expr = ' AND '.join([sponsors_expr, status_expr, date_expr])
# search_expr

#### URL encode: 

In [9]:
# search_expr_url_encode = urllib.parse.quote_plus(search_expr)
# search_expr_url_encode

#### Fields:

Les champs que l'on veut récupérer:

In [10]:
# fields = [
#     'NCTId',
#     'BriefTitle',
#     'OverallStatus',
#     'StudyType',
#     'LeadSponsorName',
#     'CollaboratorName',
#     'OrgStudyId',
#     'SecondaryId',
#     'StudyFirstPostDate',
#     'ReferencePMID',
#     'ReferenceCitation',
#     'ReferenceType',
# ]

### Envoi de la requête :

In [11]:
# study_fields = ct.get_study_fields(
#     search_expr=search_expr_url_encode,
#     fields=fields,
#     max_studies=1000,
#     fmt='csv',
# )

In [12]:
# print(f'NStudiesReturned: {len(study_fields[1:])}')

### Lire le résultat de la requête dans Pandas :

In [13]:
# import pandas as pd

In [14]:
# pd.DataFrame.from_records(study_fields[1:], index='Rank', columns=study_fields[0])

## API v2:

L'API v1 ne sera plus supporté a [partir de mi-2024](https://clinicaltrials.gov/data-api/api) :

>***Notice to API users:  
>The new ClinicalTrials.gov API, version 2.0 is available. Classic API users are strongly encouraged to switch to the modernized API. We will continue to support the classic API until mid-2024 and are planning blackouts for the spring to help with the transition to the modernized API.***

De plus, l'API v2 supporte un nouveau champs **"HasResults"**, qui pour l'instant n'est que très peu utilisé mais qui pourrait être utile à l'avenir.

En contre-partie, l'export des données sous forme de CSV est limité à un certain nombre de champs visible sur cette page: https://clinicaltrials.gov/data-api/about-api/csv-download

On est donc obligé d'utiliser l'export de données sous forme de JSON.

In [15]:
import urllib.parse

### Création de la requête:

`pytrials` n'étant pas compatible avec la v2, on envoie la requête manuellement en utilisant [Requests](https://requests.readthedocs.io/en/latest/) 

#### Format:

In [16]:
format = 'json'

#### Sponsors:

In [17]:
sponsors = [
    'anrs',
    'inserm',
    'institut national de la santé et de la recherche médicale',
    'french national agency for research on aids and viral hepatitis',
]

In [18]:
sponsors_expr_v2 = ' OR '.join(sponsors)
sponsors_expr_v2 = urllib.parse.quote_plus(sponsors_expr_v2)
sponsors_expr_v2

'anrs+OR+inserm+OR+institut+national+de+la+sant%C3%A9+et+de+la+recherche+m%C3%A9dicale+OR+french+national+agency+for+research+on+aids+and+viral+hepatitis'

#### Overall_status:

In [19]:
overall_status = 'COMPLETED'

#### Fields:

In [20]:
fields_v2 = [
    'NCTId',
    'BriefTitle',
    # 'OfficialTitle',
    'OverallStatus',
    'StudyType',
    'LeadSponsorName',
    'CollaboratorName',
    # 'OrgStudyId',
    # 'SecondaryId',
    'StudyFirstPostDate',
    'StartDate',
    # 'PrimaryCompletionDate',
    'CompletionDate',
    'ReferencePMID',
    'ReferenceCitation',
    'ReferenceType',
    'hasResults',
]
fields_v2

['NCTId',
 'BriefTitle',
 'OverallStatus',
 'StudyType',
 'LeadSponsorName',
 'CollaboratorName',
 'StudyFirstPostDate',
 'StartDate',
 'CompletionDate',
 'ReferencePMID',
 'ReferenceCitation',
 'ReferenceType',
 'hasResults']

In [21]:
fields_expr_v2 = ','.join(fields_v2)
fields_expr_v2 = urllib.parse.quote_plus(fields_expr_v2)
fields_expr_v2

'NCTId%2CBriefTitle%2COverallStatus%2CStudyType%2CLeadSponsorName%2CCollaboratorName%2CStudyFirstPostDate%2CStartDate%2CCompletionDate%2CReferencePMID%2CReferenceCitation%2CReferenceType%2ChasResults'

#### Date de fin d'étude postérieure à 2013:

<span style="color:red">**Mettre à jour la date si nécessaire**</span>

In [22]:
date = '01/01/2013'
date_expr = urllib.parse.quote_plus(f'AREA[CompletionDate]RANGE[{date}, MAX]')
date_expr

'AREA%5BCompletionDate%5DRANGE%5B01%2F01%2F2013%2C+MAX%5D'

#### Nombre de résultats max :

In [23]:
count_total = 'true'

In [24]:
page_size = 1000

#### URL api:

In [25]:
# Make sure we don't have more than 1000 results, otherwise we need to handle several page of results
query_url = f'https://clinicaltrials.gov/api/v2/studies?format={format}&query.lead={sponsors_expr_v2}&filter.overallStatus={overall_status}&fields={fields_expr_v2}&filter.advanced={date_expr}&countTotal={count_total}&pageSize={page_size}'
query_url

'https://clinicaltrials.gov/api/v2/studies?format=json&query.lead=anrs+OR+inserm+OR+institut+national+de+la+sant%C3%A9+et+de+la+recherche+m%C3%A9dicale+OR+french+national+agency+for+research+on+aids+and+viral+hepatitis&filter.overallStatus=COMPLETED&fields=NCTId%2CBriefTitle%2COverallStatus%2CStudyType%2CLeadSponsorName%2CCollaboratorName%2CStudyFirstPostDate%2CStartDate%2CCompletionDate%2CReferencePMID%2CReferenceCitation%2CReferenceType%2ChasResults&filter.advanced=AREA%5BCompletionDate%5DRANGE%5B01%2F01%2F2013%2C+MAX%5D&countTotal=true&pageSize=1000'

### Envoi de la requête :

Installer [Requests](https://requests.readthedocs.io/en/latest/) et [Requests_cache](https://requests-cache.readthedocs.io/en/stable/index.html), si ils ne sont pas déjà présent:

`python -m pip install requests`  
`python -m pip install requests-cache`

In [26]:
from datetime import timedelta
import requests
from requests_cache import CachedSession

# Add caching to all requests functions using the session
session = CachedSession(
    cache_control = True, # Use Cache-Control response headers for expiration, if available
    expire_after = timedelta(days=1), # Otherwise expire responses after one day
)

In [27]:
response = session.get(query_url)
response.raise_for_status()
# response

In [28]:
print(f'Studies returned: {response.json()["totalCount"]}')

Studies returned: 201


### Traitement du JSON retourné:

In [29]:
import json

In [30]:
# print(json.dumps(response.json(), indent=2))

***La structure du JSON est bien trop imbriquée pour le normaliser avec Pandas, du coup on l'applatit à la main:***

A partir du JSON on construit un dictionnaire équivalent mais beaucoup plus "plat"

In [31]:
# Si la liste des collaborateurs est vide on renvoie None, sinon on concatène les valeurs de la liste
# sous la forme "collaborateur_0 | collaborateur_1 | ..."
def concatenate_collaborator_list(collaborator_list):
    if collaborator_list == []:
        return None
    else:
        return ' | '.join(collaborator_list)

In [32]:
studies_list = []
for study in response.json()['studies']:
    study_pro = study['protocolSection']
    study_dict = {
        'NCTId': study_pro['identificationModule']['nctId'],
        'BriefTitle': study_pro['identificationModule']['briefTitle'],
        'LeadSponsorName': (
            study_pro['sponsorCollaboratorsModule']['leadSponsor']['name']
        ),
        'CollaboratorName': concatenate_collaborator_list(
            [
                c['name'] for c in (
                    study_pro['sponsorCollaboratorsModule'].get('collaborators', [])  # can be missing
                )
            ]
        ),
        'OverallStatus': study_pro['statusModule']['overallStatus'],
        'StudyType': study_pro['designModule']['studyType'],
        'HasResults': study['hasResults'],
        'StudyFirstPostDate': (
            study_pro['statusModule']['studyFirstPostDateStruct']['date']
        ),
        'StartDate': (
            study_pro['statusModule'].get('startDateStruct', {}).get('date', None) # can be missing
        ),
        # 'PrimaryCompletionDate' : study_pro["statusModule"].get('primaryCompletionDateStruct', {}).get('date', None), # can be missing
        'CompletionDate': (
            study_pro['statusModule'].get('completionDateStruct', {}).get('date', None) # can be missing
        ),
        'Reference': (
            study_pro.get('referencesModule', {}).get('references', []) # can be missing
        ),
    }
    studies_list.append(study_dict)

# print(json.dumps(studies_list, indent=2))

On vérifie que l'on n'a pas perdu de NCTId en route:

In [33]:
print(f'Nombre de NCTId: {len(studies_list)}')
assert response.json()['totalCount'] == len(studies_list)

Nombre de NCTId: 201


### Import dans Pandas

In [34]:
import pandas as pd

In [35]:
df_ct = pd.json_normalize(data=studies_list)
df_ct

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,Reference
0,NCT00658346,Molecular Diversity of HIV-1 Group O Strains a...,"ANRS, Emerging Infectious Diseases",None,COMPLETED,OBSERVATIONAL,False,2008-04-15,2010-06,2016-09,"[{'pmid': '31063537', 'type': 'DERIVED', 'cita..."
1,NCT01647295,Social Interactions: Ocular Explorations and P...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,OBSERVATIONAL,False,2012-07-23,2013-02-25,2019-06-18,[]
2,NCT01332955,Telaprevir in HIV-HCV Coinfected Patients Who ...,"ANRS, Emerging Infectious Diseases",Janssen-Cilag Ltd.,COMPLETED,INTERVENTIONAL,False,2011-04-11,2011-04,2013-09,"[{'pmid': '25139963', 'type': 'DERIVED', 'cita..."
3,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,"[{'pmid': '15037514', 'type': 'RESULT', 'citat..."
4,NCT02488395,Superior Colliculus Activity in Parkinson Dise...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,OBSERVATIONAL,False,2015-07-02,2015-10-07,2018-01-19,"[{'pmid': '22639796', 'type': 'RESULT', 'citat..."
...,...,...,...,...,...,...,...,...,...,...,...
196,NCT02172677,The Influence of Collective Schemas on Individ...,Institut National de la Santé Et de la Recherc...,La Région Basse-Normandie | Université de Caen...,COMPLETED,INTERVENTIONAL,False,2014-06-24,2014-10,2016-10-14,[]
197,NCT02481453,Rapamycine vs Placebo for the Treatment of Inc...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2015-06-25,2015-07-15,2018-01-22,"[{'pmid': '20413309', 'type': 'BACKGROUND', 'c..."
198,NCT02051413,Beta-arrestins and Response to Venlafaxine in ...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2014-01-31,2014-02-18,2018-05,[]
199,NCT02014883,Phase II Open Label Study Using Triheptanoin i...,Institut National de la Santé Et de la Recherc...,Ultragenyx Pharmaceutical Inc,COMPLETED,INTERVENTIONAL,False,2013-12-18,2013-12-04,2019-07-04,"[{'pmid': '26536893', 'type': 'DERIVED', 'cita..."


#### On "explose" la colonne "References":

Pour chaque NCTId, la colonne réferences contient potentiellement une liste de plusieurs références.  
Si par exemple, on a 3 références, on veut se retrouver avec 3 lignes, chacune indexée par le même NCTId et contenant une unique réference.  

*Avant*:  
**`NCTId    References`**  
`NCT0001, [Ref1, Ref2, Ref3]`   

*Après*:  
**`NCTId    References`**  
`NCT0001,   Ref1`  
`NCT0001,   Ref2`  
`NCT0001,   Ref3`   

In [36]:
df_ct = df_ct.explode('Reference', ignore_index=True)
df_ct.loc[:, ['NCTId', 'Reference']]

,NCTId,Reference
0,NCT00658346,"{'pmid': '31063537', 'type': 'DERIVED', 'citat..."
1,NCT01647295,NaN
2,NCT01332955,"{'pmid': '25139963', 'type': 'DERIVED', 'citat..."
3,NCT01432821,"{'pmid': '15037514', 'type': 'RESULT', 'citati..."
4,NCT01432821,"{'pmid': '15777732', 'type': 'RESULT', 'citati..."
...,...,...
478,NCT02481453,"{'pmid': '24975859', 'type': 'BACKGROUND', 'ci..."
479,NCT02051413,NaN
480,NCT02014883,"{'pmid': '26536893', 'type': 'DERIVED', 'citat..."
481,NCT02658253,"{'pmid': '33717176', 'type': 'DERIVED', 'citat..."


Pour chaque NCTId, la colonne réference contient maintenant un dictionnaire de la forme:  
`{  
"pmid": "17545707",  
"type": "BACKGROUND",  
"citation": "...",  
}`

Dont on veut extraire de nouvelles colonnes en utilant les clés du dictionnaire:  
**`pmid     type        citation`**  
`17545707, BACKGROUND, "..."`

In [37]:
df_ct_references = pd.json_normalize(df_ct.pop('Reference'))
df_ct_references

,pmid,type,citation
0,31063537,DERIVED,"Kouanfack C, Unal G, Schaeffer L, Kfutwah A, A..."
1,NaN,NaN,NaN
2,25139963,DERIVED,"Cotte L, Braun J, Lascoux-Combe C, Vincent C, ..."
3,15037514,RESULT,"Ahmad S, Fowler LJ, Whitton PS. Effect of acut..."
4,15777732,RESULT,"Ahmad S, Fowler LJ, Whitton PS. Lamotrigine, c..."
...,...,...,...
478,24975859,BACKGROUND,"Lloyd TE, Mammen AL, Amato AA, Weiss MD, Needh..."
479,NaN,NaN,NaN
480,26536893,DERIVED,"Mochel F, Hainque E, Gras D, Adanyeguh IM, Cai..."
481,33717176,DERIVED,"Gamain B, Chene A, Viebig NK, Tuikue Ndam N, N..."


On réassemble la dataFrame complète:

In [38]:
df_ct = df_ct.join(
    df_ct_references,
    validate = 'one_to_one',
)
df_ct

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,pmid,type,citation
0,NCT00658346,Molecular Diversity of HIV-1 Group O Strains a...,"ANRS, Emerging Infectious Diseases",None,COMPLETED,OBSERVATIONAL,False,2008-04-15,2010-06,2016-09,31063537,DERIVED,"Kouanfack C, Unal G, Schaeffer L, Kfutwah A, A..."
1,NCT01647295,Social Interactions: Ocular Explorations and P...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,OBSERVATIONAL,False,2012-07-23,2013-02-25,2019-06-18,NaN,NaN,NaN
2,NCT01332955,Telaprevir in HIV-HCV Coinfected Patients Who ...,"ANRS, Emerging Infectious Diseases",Janssen-Cilag Ltd.,COMPLETED,INTERVENTIONAL,False,2011-04-11,2011-04,2013-09,25139963,DERIVED,"Cotte L, Braun J, Lascoux-Combe C, Vincent C, ..."
3,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,15037514,RESULT,"Ahmad S, Fowler LJ, Whitton PS. Effect of acut..."
4,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,15777732,RESULT,"Ahmad S, Fowler LJ, Whitton PS. Lamotrigine, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,NCT02481453,Rapamycine vs Placebo for the Treatment of Inc...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2015-06-25,2015-07-15,2018-01-22,24975859,BACKGROUND,"Lloyd TE, Mammen AL, Amato AA, Weiss MD, Needh..."
479,NCT02051413,Beta-arrestins and Response to Venlafaxine in ...,Institut National de la Santé Et de la Recherc...,None,COMPLETED,INTERVENTIONAL,False,2014-01-31,2014-02-18,2018-05,NaN,NaN,NaN
480,NCT02014883,Phase II Open Label Study Using Triheptanoin i...,Institut National de la Santé Et de la Recherc...,Ultragenyx Pharmaceutical Inc,COMPLETED,INTERVENTIONAL,False,2013-12-18,2013-12-04,2019-07-04,26536893,DERIVED,"Mochel F, Hainque E, Gras D, Adanyeguh IM, Cai..."
481,NCT02658253,Trial to Evaluate the Safety and Immunogenicit...,Institut National de la Santé Et de la Recherc...,"EVI Industries, Inc. | Recherche Clinique Pari...",COMPLETED,INTERVENTIONAL,False,2016-01-18,2016-01,2019-02-21,33717176,DERIVED,"Gamain B, Chene A, Viebig NK, Tuikue Ndam N, N..."


***On reconstruit l'index :***

In [39]:
# df_studies_v2.set_index('NCTId', inplace = True)

***On précise les types :***

In [40]:
df_ct = df_ct.convert_dtypes()
df_ct = df_ct.astype(
    {
        'OverallStatus': 'category',
        'StudyType': 'category',
        'type': 'category',
    }
)
df_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   NCTId               483 non-null    string  
 1   BriefTitle          483 non-null    string  
 2   LeadSponsorName     483 non-null    string  
 3   CollaboratorName    218 non-null    string  
 4   OverallStatus       483 non-null    category
 5   StudyType           483 non-null    category
 6   HasResults          483 non-null    boolean 
 7   StudyFirstPostDate  483 non-null    string  
 8   StartDate           483 non-null    string  
 9   CompletionDate      483 non-null    string  
 10  pmid                364 non-null    string  
 11  type                364 non-null    category
 12  citation            364 non-null    string  
dtypes: boolean(1), category(3), string(9)
memory usage: 36.8 KB


In [41]:
df_ct

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,pmid,type,citation
0,NCT00658346,Molecular Diversity of HIV-1 Group O Strains a...,"ANRS, Emerging Infectious Diseases",<NA>,COMPLETED,OBSERVATIONAL,False,2008-04-15,2010-06,2016-09,31063537,DERIVED,"Kouanfack C, Unal G, Schaeffer L, Kfutwah A, A..."
1,NCT01647295,Social Interactions: Ocular Explorations and P...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,OBSERVATIONAL,False,2012-07-23,2013-02-25,2019-06-18,<NA>,<NA>,<NA>
2,NCT01332955,Telaprevir in HIV-HCV Coinfected Patients Who ...,"ANRS, Emerging Infectious Diseases",Janssen-Cilag Ltd.,COMPLETED,INTERVENTIONAL,False,2011-04-11,2011-04,2013-09,25139963,DERIVED,"Cotte L, Braun J, Lascoux-Combe C, Vincent C, ..."
3,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,15037514,RESULT,"Ahmad S, Fowler LJ, Whitton PS. Effect of acut..."
4,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,15777732,RESULT,"Ahmad S, Fowler LJ, Whitton PS. Lamotrigine, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,NCT02481453,Rapamycine vs Placebo for the Treatment of Inc...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2015-06-25,2015-07-15,2018-01-22,24975859,BACKGROUND,"Lloyd TE, Mammen AL, Amato AA, Weiss MD, Needh..."
479,NCT02051413,Beta-arrestins and Response to Venlafaxine in ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2014-01-31,2014-02-18,2018-05,<NA>,<NA>,<NA>
480,NCT02014883,Phase II Open Label Study Using Triheptanoin i...,Institut National de la Santé Et de la Recherc...,Ultragenyx Pharmaceutical Inc,COMPLETED,INTERVENTIONAL,False,2013-12-18,2013-12-04,2019-07-04,26536893,DERIVED,"Mochel F, Hainque E, Gras D, Adanyeguh IM, Cai..."
481,NCT02658253,Trial to Evaluate the Safety and Immunogenicit...,Institut National de la Santé Et de la Recherc...,"EVI Industries, Inc. | Recherche Clinique Pari...",COMPLETED,INTERVENTIONAL,False,2016-01-18,2016-01,2019-02-21,33717176,DERIVED,"Gamain B, Chene A, Viebig NK, Tuikue Ndam N, N..."


#### Export en CSV :

In [42]:
# df_ct.to_csv(
#     'Data/outputs/extract_CT_api_v2.csv',
#     sep=';',
#     index=False,
#     encoding='utf-8-sig',
# )

# PubMed

### Utilisation d'une clé pour l'API Pubmed : 

Il est recommandé d'utiliser une clé pour accéder à l'API Pubmed, ce qui permet de lancer jusqu'à 10 requêtes par seconde.  
Sans clé, la limite est de 3 requêtes par seconde.  

> E-utils users are allowed 3 requests/second without an API key. Create an API key to increase your e-utils limit to 10 requests/second.

**En pratique, l'API Pubmed étant beaucoup plus lente, cela ne semble pas changer grand chose.**

Pour récupérer sa clé, se rendre sur la page en étant loggué:
https://account.ncbi.nlm.nih.gov/settings/

Une fois la clé récupérée, l'ajouter aux variables d'environnement avec la commande suivante dans un Terminal:

**Windows :** 

`setx NCBI_API_KEY “123456"`

**Linux/MacOS :**

`export NCBI_API_KEY=123456`

In [43]:
import os

assert os.getenv('NCBI_API_KEY', None) is not None

### Récupération des PMIDs via PubMed:

Pour chaque NCTId de CT, on récupère les PMIDs des publications associées via PubMed:

In [44]:
# Liste unique des NCTId extraits de ClinicalTrial
nctids = df_ct.loc[:, 'NCTId'].unique()
num_nctids = len(nctids)

pmids_pubmed_dict = {}
for i, nctid in enumerate(nctids):
    # Display the progress on a single line
    print(f'\r{i+1}/{num_nctids}...', end='', flush=True)

    # Query Pubmed's API
    query_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&term={nctid}&api_key={os.getenv('NCBI_API_KEY')}'
    response = session.get(query_url)
    response.raise_for_status()

    pmids = response.json()['esearchresult']['idlist']
    pmids_pubmed_dict[nctid] = set(pmids)

201/201...

In [45]:
# pmids_pubmed_dict

**On veut fusionner la liste PMIDs que l'on vient de récupérer sur PubMed, à la liste des PMIDs déjà récupérés via CT.**

On met les PMIDs de CT sous la même forme:

In [46]:
pmids_ct_dict = {}
for nctid in nctids:
    pmids = df_ct[df_ct.loc[:, 'NCTId'] == nctid].loc[:, 'pmid'].dropna()
    pmids_ct_dict[nctid] = set(pmids)
# pmids_ct_dict

Pour un NCTDId donné, on fait l'union des deux ensembles de PMIDs:

In [47]:
pmids_complete_dict = {}
for nctid in nctids:
    # L'ensemble des PMIds présents dans PubMed et CT
    pmids_complete_dict[nctid] = pmids_pubmed_dict[nctid] | pmids_ct_dict[nctid]
# pmids_complete_dict

#### Verifications:

In [48]:
num_pmids_ct = sum((len(v) for v in pmids_ct_dict.values()))
print(f'Nombre total de publications issus de CT: {num_pmids_ct}')

Nombre total de publications issus de CT: 364


In [49]:
num_pmids_complete = sum((len(v) for v in pmids_complete_dict.values()))
print(f'Nombre total de publications après consultation PubMed: {num_pmids_complete}')

Nombre total de publications après consultation PubMed: 395


In [50]:
pmids_pubmed_only_dict = {}
for nctid in nctids:
    # L'ensemble des PMIds présents dans PubMed uniquement
    pmids_pubmed_only_dict[nctid] = pmids_pubmed_dict[nctid] - pmids_ct_dict[nctid]

In [51]:
num_pmids_pubmed_only = sum((len(v) for v in pmids_pubmed_only_dict.values()))
print(f'Nombre de nouveaux PMIDs trouvés via Pubmed: {num_pmids_pubmed_only}')

Nombre de nouveaux PMIDs trouvés via Pubmed: 31


In [52]:
assert num_pmids_complete - num_pmids_ct == num_pmids_pubmed_only

In [53]:
print('NCTId des nouveaux PMIDs trouvés via Pubmed:')
{k: v for k, v in pmids_pubmed_only_dict.items() if v != set()}

NCTId des nouveaux PMIDs trouvés via Pubmed:


{'NCT01703962': {'37668523'},
 'NCT02777229': {'37851566', '38156046'},
 'NCT05349162': {'36735263'},
 'NCT01473472': {'36601747'},
 'NCT01453192': {'30688008'},
 'NCT05311865': {'36438274', '37795682'},
 'NCT03078439': {'38408861'},
 'NCT00640263': {'34425825'},
 'NCT04315948': {'36695483', '38552208'},
 'NCT03335995': {'37497675'},
 'NCT02212379': {'31269208'},
 'NCT04945655': {'38780943'},
 'NCT01426243': {'26314624'},
 'NCT03870438': {'38484756'},
 'NCT02057796': {'36883573'},
 'NCT02405013': {'36686592'},
 'NCT01801618': {'29662875'},
 'NCT03215732': {'37143029'},
 'NCT03005652': {'38100477'},
 'NCT02833961': {'36318030'},
 'NCT04392388': {'34293141'},
 'NCT02150993': {'38740027'},
 'NCT04288128': {'38419144', '38421662'},
 'NCT01089387': {'26439886'},
 'NCT04409405': {'38043556'},
 'NCT01688453': {'35272723'},
 'NCT02481453': {'38273639'}}

In [54]:
num_nctid_empty_ct = sum((1 for v in pmids_ct_dict.values() if v == set()))
print(f"Nombre d'études sans PMIDs issus de CT: {num_nctid_empty_ct}")

Nombre d'études sans PMIDs issus de CT: 119


In [55]:
num_nctid_empty_pubmed = sum((1 for v in pmids_complete_dict.values() if v == set()))
print(f"Nombre d'études sans PMIDs après consultation PubMed: {num_nctid_empty_pubmed}")

Nombre d'études sans PMIDs après consultation PubMed: 106


In [56]:
print("NCTIds qui n'avait aucun PMIDs sous CT, mais enrichis via Pubmed:")
nctids_previously_empty = (
    {k for k, v in pmids_ct_dict.items() if v == set()}
    - {k for k, v in pmids_complete_dict.items() if v == set()}
)
{k: pmids_complete_dict[k] for k in nctids_previously_empty}

NCTIds qui n'avait aucun PMIDs sous CT, mais enrichis via Pubmed:


{'NCT02833961': {'36318030'},
 'NCT04409405': {'38043556'},
 'NCT01801618': {'29662875'},
 'NCT05311865': {'36438274', '37795682'},
 'NCT01453192': {'30688008'},
 'NCT05349162': {'36735263'},
 'NCT04288128': {'38419144', '38421662'},
 'NCT01703962': {'37668523'},
 'NCT04392388': {'34293141'},
 'NCT02150993': {'38740027'},
 'NCT02212379': {'31269208'},
 'NCT03078439': {'38408861'},
 'NCT02405013': {'36686592'}}

In [57]:
len(nctids_previously_empty)

13

In [58]:
assert num_nctid_empty_ct - num_nctid_empty_pubmed == len(nctids_previously_empty)

### Enrichissement des PMIDs via l'API Pubmed

Pour chaque PMID récupéré, on l'enrichit avec les données de PubMed tel que le titre, les auteurs, ...:

In [59]:
# Si la liste des auteurs est vide on renvoie None, sinon on concatène les valeurs de la liste
# sous la forme "auteur0; auteur1 ..."
def concatenate_authors_list(authors_list):
    if authors_list == []:
        return None
    else:
        return '; '.join([author['name'] for author in authors_list])

In [60]:
# We return a list of dictionnaries, each dictionnary containing the NCTId and the publication associated with it
# total_publications_list = [{'NCTId': '...', 'publications': [{'pmid': '...', 'title': '...'}, {'pmid': '...', 'title': '...'}]}, {'NCTId': ...}]
total_publications_list = []

books_list = []
counter = 0  # To keep track of progress of pmids

# For each NTCID...
for nctid, pmids in pmids_complete_dict.items():
    # [{'pmid': '...', 'title': '...'}, {'pmid': '...', 'title': '...'}]
    publications_list = []

    # If the set of pmids is not empty
    if pmids != set():
        # Query Pubmed's API with several pmids at the same time
        pmids_str = ','.join(pmids)
        query_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&retmode=json&id={pmids_str}&api_key={os.getenv('NCBI_API_KEY')}'
        response = session.get(query_url)
        response.raise_for_status()
        result = response.json()['result']

        # We process each PMID...
        for pmid in result['uids']:
            # Check that the pmid returned by Pubmed was part of the query
            assert pmid in pmids

            publication = result[pmid]

            # Display the progress on a single line
            print(f'\r{counter+1} / {num_pmids_complete}...', end='', flush=True)

            # We are not interested by articles with type 'book'
            # TODO: book special case ?
            if publication['doctype'] == 'citation':

                # Find the DOI amongst the different ids
                doi = None
                for id in publication['articleids']:
                    if id['idtype'] == 'doi':
                        doi = id['value']
                        break

                # Add the infos for the publication
                publications_list.append(
                    {
                        'pmid': pmid,
                        'title': publication['title'],
                        'authors': concatenate_authors_list(publication['authors']),
                        'doi': doi,
                        # 'date': publication['sortpubdate'], # We use Unpaywall 'published_date' instead
                        'publication_types': publication['pubtype'],
                    }
                )
            else:
                # type different from 'article', gets ignored
                books_list.append((nctid, pmid))

            counter += 1

    publication_dict = {'NCTId': nctid, 'publications': publications_list}
    total_publications_list.append(publication_dict)

# total_publications_list

395 / 395...

In [61]:
print(books_list)

[('NCT03537196', '27227200')]


In [62]:
# print(json.dumps(total_publications_list, indent=2))

Publications sans DOI:

In [63]:
# PMID without DOI
for study in total_publications_list:
    for publi in study['publications']:
        if publi['doi'] is None:
            print(f'PMID without DOI: {publi['pmid']}')

PMID without DOI: 19839502


In [64]:
# The number of NCTId didn't change
assert len(total_publications_list) == len(studies_list)

### Import dans Pandas

In [65]:
df_pubmed = pd.DataFrame.from_records(total_publications_list)
df_pubmed

,NCTId,publications
0,NCT00658346,"[{'pmid': '31063537', 'title': 'Comparative Im..."
1,NCT01647295,[]
2,NCT01332955,"[{'pmid': '25139963', 'title': 'Telaprevir for..."
3,NCT01432821,"[{'pmid': '2271377', 'title': 'Apomorphine-ind..."
4,NCT02488395,"[{'pmid': '23916713', 'title': 'Enhanced visua..."
...,...,...
196,NCT02172677,[]
197,NCT02481453,"[{'pmid': '38273639', 'title': 'Sirolimus for ..."
198,NCT02051413,[]
199,NCT02014883,"[{'pmid': '26536893', 'title': 'Triheptanoin d..."


#### On "explose" la colonne "publications":

Pour chaque NCTId, la colonne 'publications' contient potentiellement une liste de plusieurs publications.  
Si par exemple, on a 3 publications, on veut se retrouver avec 3 lignes, chacune indexée par le même NCTId et contenant une unique publication.  

*Avant*:  
**`NCTId    Reference`**  
`NCT0001, [Pub1, Pub2, Pub3]`   

*Après*:  
**`NCTId    Reference`**  
`NCT0001,   Pub1`  
`NCT0001,   Pub2`  
`NCT0001,   Pub3`   

In [66]:
df_pubmed = df_pubmed.explode('publications', ignore_index=True)
df_pubmed

,NCTId,publications
0,NCT00658346,"{'pmid': '31063537', 'title': 'Comparative Imm..."
1,NCT01647295,NaN
2,NCT01332955,"{'pmid': '25139963', 'title': 'Telaprevir for ..."
3,NCT01432821,"{'pmid': '2271377', 'title': 'Apomorphine-indu..."
4,NCT01432821,"{'pmid': '15037514', 'title': 'Effect of acute..."
...,...,...
495,NCT02481453,"{'pmid': '24975859', 'title': 'Evaluation and ..."
496,NCT02051413,NaN
497,NCT02014883,"{'pmid': '26536893', 'title': 'Triheptanoin dr..."
498,NCT02658253,"{'pmid': '33717176', 'title': 'Progress and In..."


On vérifie que l'on a retrouvé plus de publications avec PubMed + CT que CT tout seul:

In [67]:
assert len(df_pubmed) >= len(df_ct)

Pour chaque NCTId, la colonne 'publications' contient maintenant un dictionnaire de la forme:  
`{  
"pmid": "17545707",  
"title": "Haematological ...",  
"authors": "Smith DJ; ...",  
}`

Dont on veut extraire de nouvelles colonnes en utilant les clés du dictionnaire:  
**`pmid      title                 authors`**  
`17545707, "Haematological ...", "Smith DJ; ..."`

In [68]:
df_pubmed_publications = pd.json_normalize(df_pubmed.pop('publications'))
df_pubmed_publications

,pmid,title,authors,doi,publication_types
0,31063537,Comparative Immunovirological and Clinical Res...,Kouanfack C; Unal G; Schaeffer L; Kfutwah A; A...,10.1093/cid/ciz371,[Journal Article]
1,NaN,NaN,NaN,NaN,NaN
2,25139963,Telaprevir for HIV/hepatitis C virus-coinfecte...,Cotte L; Braun J; Lascoux-Combe C; Vincent C; ...,10.1093/cid/ciu659,[Journal Article]
3,2271377,Apomorphine-induced blinking and yawning in he...,Blin O; Masson G; Azulay JP; Fondarai J; Serra...,10.1111/j.1365-2125.1990.tb03848.x,"[Clinical Trial, Journal Article, Randomized C..."
4,15037514,Effect of acute and chronic lamotrigine on bas...,Ahmad S; Fowler LJ; Whitton PS,10.1038/sj.bjp.0705737,[Journal Article]
...,...,...,...,...,...
495,24975859,Evaluation and construction of diagnostic crit...,Lloyd TE; Mammen AL; Amato AA; Weiss MD; Needh...,10.1212/WNL.0000000000000642,"[Journal Article, Review]"
496,NaN,NaN,NaN,NaN,NaN
497,26536893,Triheptanoin dramatically reduces paroxysmal m...,Mochel F; Hainque E; Gras D; Adanyeguh IM; Cai...,10.1136/jnnp-2015-311475,"[Clinical Trial, Journal Article]"
498,33717176,Progress and Insights Toward an Effective Plac...,Gamain B; Chêne A; Viebig NK; Tuikue Ndam N; N...,10.3389/fimmu.2021.634508,"[Journal Article, Review]"


On réassemble la dataFrame complète:

In [69]:
df_pubmed = df_pubmed.join(
    df_pubmed_publications,
    validate = 'one_to_one',
)
df_pubmed

,NCTId,pmid,title,authors,doi,publication_types
0,NCT00658346,31063537,Comparative Immunovirological and Clinical Res...,Kouanfack C; Unal G; Schaeffer L; Kfutwah A; A...,10.1093/cid/ciz371,[Journal Article]
1,NCT01647295,NaN,NaN,NaN,NaN,NaN
2,NCT01332955,25139963,Telaprevir for HIV/hepatitis C virus-coinfecte...,Cotte L; Braun J; Lascoux-Combe C; Vincent C; ...,10.1093/cid/ciu659,[Journal Article]
3,NCT01432821,2271377,Apomorphine-induced blinking and yawning in he...,Blin O; Masson G; Azulay JP; Fondarai J; Serra...,10.1111/j.1365-2125.1990.tb03848.x,"[Clinical Trial, Journal Article, Randomized C..."
4,NCT01432821,15037514,Effect of acute and chronic lamotrigine on bas...,Ahmad S; Fowler LJ; Whitton PS,10.1038/sj.bjp.0705737,[Journal Article]
...,...,...,...,...,...,...
495,NCT02481453,24975859,Evaluation and construction of diagnostic crit...,Lloyd TE; Mammen AL; Amato AA; Weiss MD; Needh...,10.1212/WNL.0000000000000642,"[Journal Article, Review]"
496,NCT02051413,NaN,NaN,NaN,NaN,NaN
497,NCT02014883,26536893,Triheptanoin dramatically reduces paroxysmal m...,Mochel F; Hainque E; Gras D; Adanyeguh IM; Cai...,10.1136/jnnp-2015-311475,"[Clinical Trial, Journal Article]"
498,NCT02658253,33717176,Progress and Insights Toward an Effective Plac...,Gamain B; Chêne A; Viebig NK; Tuikue Ndam N; N...,10.3389/fimmu.2021.634508,"[Journal Article, Review]"


### Jointure des DataFrame de CT et Pubmed:

In [70]:
df_final = df_ct.merge(
    df_pubmed, 
    on = ['NCTId', 'pmid'], 
    how = 'right',
    validate = 'one_to_one',
)

# Remove 'citation' column
df_final = df_final.drop(columns='citation')

df_final

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,pmid,type,title,authors,doi,publication_types
0,NCT00658346,Molecular Diversity of HIV-1 Group O Strains a...,"ANRS, Emerging Infectious Diseases",<NA>,COMPLETED,OBSERVATIONAL,False,2008-04-15,2010-06,2016-09,31063537,DERIVED,Comparative Immunovirological and Clinical Res...,Kouanfack C; Unal G; Schaeffer L; Kfutwah A; A...,10.1093/cid/ciz371,[Journal Article]
1,NCT01647295,Social Interactions: Ocular Explorations and P...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,OBSERVATIONAL,False,2012-07-23,2013-02-25,2019-06-18,<NA>,<NA>,NaN,NaN,NaN,NaN
2,NCT01332955,Telaprevir in HIV-HCV Coinfected Patients Who ...,"ANRS, Emerging Infectious Diseases",Janssen-Cilag Ltd.,COMPLETED,INTERVENTIONAL,False,2011-04-11,2011-04,2013-09,25139963,DERIVED,Telaprevir for HIV/hepatitis C virus-coinfecte...,Cotte L; Braun J; Lascoux-Combe C; Vincent C; ...,10.1093/cid/ciu659,[Journal Article]
3,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,2271377,RESULT,Apomorphine-induced blinking and yawning in he...,Blin O; Masson G; Azulay JP; Fondarai J; Serra...,10.1111/j.1365-2125.1990.tb03848.x,"[Clinical Trial, Journal Article, Randomized C..."
4,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,15037514,RESULT,Effect of acute and chronic lamotrigine on bas...,Ahmad S; Fowler LJ; Whitton PS,10.1038/sj.bjp.0705737,[Journal Article]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NCT02481453,Rapamycine vs Placebo for the Treatment of Inc...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2015-06-25,2015-07-15,2018-01-22,24975859,BACKGROUND,Evaluation and construction of diagnostic crit...,Lloyd TE; Mammen AL; Amato AA; Weiss MD; Needh...,10.1212/WNL.0000000000000642,"[Journal Article, Review]"
496,NCT02051413,Beta-arrestins and Response to Venlafaxine in ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2014-01-31,2014-02-18,2018-05,<NA>,<NA>,NaN,NaN,NaN,NaN
497,NCT02014883,Phase II Open Label Study Using Triheptanoin i...,Institut National de la Santé Et de la Recherc...,Ultragenyx Pharmaceutical Inc,COMPLETED,INTERVENTIONAL,False,2013-12-18,2013-12-04,2019-07-04,26536893,DERIVED,Triheptanoin dramatically reduces paroxysmal m...,Mochel F; Hainque E; Gras D; Adanyeguh IM; Cai...,10.1136/jnnp-2015-311475,"[Clinical Trial, Journal Article]"
498,NCT02658253,Trial to Evaluate the Safety and Immunogenicit...,Institut National de la Santé Et de la Recherc...,"EVI Industries, Inc. | Recherche Clinique Pari...",COMPLETED,INTERVENTIONAL,False,2016-01-18,2016-01,2019-02-21,33717176,DERIVED,Progress and Insights Toward an Effective Plac...,Gamain B; Chêne A; Viebig NK; Tuikue Ndam N; N...,10.3389/fimmu.2021.634508,"[Journal Article, Review]"


Les nouveaux PMIDs trouvés via Pubmed, n'ont aucune des infos associées avec CT présentes: BriefTitle, LeadSponsorName, etc ...

In [71]:
# Index of empty rows we need to fill
index_empty_rows = df_final.loc[:, 'BriefTitle'].isna()

# Columns we need to fill
columns_to_fill = [
    'BriefTitle',
    'LeadSponsorName',
    'CollaboratorName',
    'OverallStatus',
    'StudyType',
    'HasResults',
    'StudyFirstPostDate',
    'StartDate',
    'CompletionDate',
    'type',
]

df_final.loc[index_empty_rows, ['NCTId'] + columns_to_fill].head(10)

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,type
36,NCT01703962,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
47,NCT02777229,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
50,NCT02777229,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
64,NCT05349162,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
71,NCT01473472,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
80,NCT01453192,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
82,NCT05311865,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
83,NCT05311865,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
89,NCT03078439,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
109,NCT00640263,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


On rajoute un type 'PUBMED' pour les PMIDs issues de PubMed uniquement:

In [72]:
# Add new 'PUBMED' category
df_final['type'] = df_final['type'].cat.add_categories('PUBMED').astype('category')

# We add a 'PUBMED' type to the PMIDs extracted from Pubmed exclusively
df_final.loc[index_empty_rows, 'type'] = 'PUBMED'

In [73]:
df_final.loc[index_empty_rows, ['NCTId'] + columns_to_fill].head(10)

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,type
36,NCT01703962,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
47,NCT02777229,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
50,NCT02777229,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
64,NCT05349162,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
71,NCT01473472,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
80,NCT01453192,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
82,NCT05311865,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
83,NCT05311865,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
89,NCT03078439,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED
109,NCT00640263,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PUBMED


On va remplir ces colonnes avec les infos contenus dans la DataFrame CT:

In [74]:
# NCTIds of empty rows
NCTIds_empty_rows = df_final.loc[index_empty_rows, 'NCTId']

# Columns we wish to copy
columns_to_copy = [
    'BriefTitle',
    'LeadSponsorName',
    'CollaboratorName',
    'OverallStatus',
    'StudyType',
    'HasResults',
    'StudyFirstPostDate',
    'StartDate',
    'CompletionDate',
]

# We copy the missing values from the CT dataframe
for index, nctid in NCTIds_empty_rows.items():
    # For an NCTId, we look in the CT dataframe for the first row with this NCTDId
    # and copy the missings columns
    df_final.loc[index, columns_to_copy] = df_ct.loc[
        df_ct.loc[:, 'NCTId'] == nctid, columns_to_copy
    ].iloc[0]

In [75]:
df_final.loc[index_empty_rows, ['NCTId'] + columns_to_fill].head(10)

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,type
36,NCT01703962,Non Invasive IDentification of Gliomas With ID...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,OBSERVATIONAL,False,2012-10-11,2012-03-14,2014-03-20,PUBMED
47,NCT02777229,Efficacy and Safety of a Dolutegravir-based Re...,"ANRS, Emerging Infectious Diseases",Institut de Recherche pour le Developpement | ...,COMPLETED,INTERVENTIONAL,False,2016-05-19,2016-07,2021-07,PUBMED
50,NCT02777229,Efficacy and Safety of a Dolutegravir-based Re...,"ANRS, Emerging Infectious Diseases",Institut de Recherche pour le Developpement | ...,COMPLETED,INTERVENTIONAL,False,2016-05-19,2016-07,2021-07,PUBMED
64,NCT05349162,Epicardial vs. Transvenous ICDs in Children,Paris Cardiovascular Research Center (Inserm U...,Hôpital Necker-Enfants Malades,COMPLETED,OBSERVATIONAL,False,2022-04-27,2003-01-01,2022-04-01,PUBMED
71,NCT01473472,On Demand Antiretroviral Pre-exposure Prophyla...,"ANRS, Emerging Infectious Diseases",<NA>,COMPLETED,INTERVENTIONAL,False,2011-11-17,2012-01,2016-12-15,PUBMED
80,NCT01453192,Renal Transplantation and Raltegravir in HIV-I...,"ANRS, Emerging Infectious Diseases",Merck Sharp & Dohme LLC,COMPLETED,INTERVENTIONAL,False,2011-10-17,2011-12,2015-11,PUBMED
82,NCT05311865,Transmission of Covid-19 During Clubbing Event...,"ANRS, Emerging Infectious Diseases",Cerballiance | Kappa Santé,COMPLETED,INTERVENTIONAL,False,2022-04-05,2021-09-04,2022-02-26,PUBMED
83,NCT05311865,Transmission of Covid-19 During Clubbing Event...,"ANRS, Emerging Infectious Diseases",Cerballiance | Kappa Santé,COMPLETED,INTERVENTIONAL,False,2022-04-05,2021-09-04,2022-02-26,PUBMED
89,NCT03078439,EPIPAGE2 Cohort Study Follow up at Five and a ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,OBSERVATIONAL,False,2017-03-13,2016-09-02,2018-01-08,PUBMED
109,NCT00640263,Comparison of Efficacy and Safety of Infant Pe...,French National Agency for Research on AIDS an...,European and Developing Countries Clinical Tri...,COMPLETED,INTERVENTIONAL,False,2008-03-21,2009-12,2014-02,PUBMED


# Verification du status Open Access via l'API d'Unpaywall:

Ne pas oublier que le DOI peut être vide :

In [76]:
mask_doi_na = df_final.loc[:, 'pmid'].notna() & df_final.loc[:, 'doi'].isna()
df_final.loc[mask_doi_na, ['NCTId', 'pmid', 'type', 'title', 'doi']]

,NCTId,pmid,type,title,doi
129,NCT03537196,19839502,BACKGROUND,High prevalence of Hepatitis C virus genotype ...,None


#### Envoi de la requête :

L'utilisation de l'API d'Unpaywall nécessite d'inclure son adresse email dans la requête:

> **Authentication**  
> Requests must include your email as a parameter at the end of the URL, like this: api.unpaywall.org/my/request?email=YOUR_EMAIL.

Pour éviter de diffuser trop largement son adresse email, plutôt que de l'inclure dans le notebook, on l'exporte comme variable d'environnement, comme on l'a fait pour la clé API Pubmed.

Ajouter `EMAIL_ADDRESS` aux variables d'environnement avec la commande suivante dans un Terminal:

**Windows :** 

`setx EMAIL_ADDRESS "unpaywall_01@example.com"`

**Linux/MacOS :**

`export EMAIL_ADDRESS=unpaywall_01@example.com`

In [77]:
assert os.getenv('EMAIL_ADDRESS', None) is not None

In [78]:
# We return a list of dictionnaries, each dictionnary containing 
# publications_list = [{'doi': ..., 'is_oa': ...}, {'doi': ..., 'is_oa': ...}]
publications_list_un = []

# Set of all the DOIs we need information for
dois = df_final['doi'].dropna().unique()
num_dois = len(dois)

for i, doi in enumerate(dois):
    # Display the progress on a single line
    print(f'\r{i+1} / {num_dois}...', end='', flush=True)

    # Query Unpaywall's API
    query_url = (
        f'https://api.unpaywall.org/v2/{doi}?email={os.getenv('EMAIL_ADDRESS')}'
    )
    response = session.get(query_url)
    response.raise_for_status()
    result = response.json()

    # Assert doi == result['doi'] ?

    # Add the info for the publication
    publications_list_un.append(
        {
            'doi': doi,
            # 'genre': result['genre'], # always 'journal-article' for now, add a check ?
            'is_paratext': result['is_paratext'],
            'published_date': result['published_date'],
            'is_oa': result['is_oa'],
            'oa_status': result['oa_status'],
            'has_repository_copy': result['has_repository_copy'],
            # add 'oa_date' to compare with 'published_date'?
        }
    )

387 / 387...

In [79]:
df_unpaywall = pd.DataFrame(publications_list_un).astype(
    {
        'is_paratext': 'boolean',
        'is_oa': 'boolean',
        'oa_status': 'category',
        'has_repository_copy': 'boolean',
    }
)
df_unpaywall

,doi,is_paratext,published_date,is_oa,oa_status,has_repository_copy
0,10.1093/cid/ciz371,False,2019-05-07,True,hybrid,False
1,10.1093/cid/ciu659,False,2014-08-18,True,bronze,False
2,10.1111/j.1365-2125.1990.tb03848.x,False,1990-11-01,True,green,True
3,10.1038/sj.bjp.0705737,False,2004-05-01,True,green,True
4,10.1212/01.wnl.0000132840.40838.13,False,2004-07-13,False,closed,False
...,...,...,...,...,...,...
382,10.1016/j.nmd.2010.03.014,False,2010-06-01,True,bronze,False
383,10.1212/WNL.0000000000000642,False,2014-07-29,True,green,True
384,10.1136/jnnp-2015-311475,False,2015-11-03,True,hybrid,True
385,10.3389/fimmu.2021.634508,False,2021-02-25,True,gold,True


In [80]:
# Assert ? Test in the loop ?
# df_unpaywall['is_paratext'].value_counts()

#### Jointure des dataframes :

In [81]:
df_with_oa_status = df_final.merge(
    df_unpaywall,
    on = ['doi'], 
    how = 'left',
    validate = 'many_to_one',
)

# df_with_oa_status.loc[:, ['NCTId', 'pmid', 'doi', 'is_oa']]
df_with_oa_status

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,...,type,title,authors,doi,publication_types,is_paratext,published_date,is_oa,oa_status,has_repository_copy
0,NCT00658346,Molecular Diversity of HIV-1 Group O Strains a...,"ANRS, Emerging Infectious Diseases",<NA>,COMPLETED,OBSERVATIONAL,False,2008-04-15,2010-06,2016-09,...,DERIVED,Comparative Immunovirological and Clinical Res...,Kouanfack C; Unal G; Schaeffer L; Kfutwah A; A...,10.1093/cid/ciz371,[Journal Article],False,2019-05-07,True,hybrid,False
1,NCT01647295,Social Interactions: Ocular Explorations and P...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,OBSERVATIONAL,False,2012-07-23,2013-02-25,2019-06-18,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>
2,NCT01332955,Telaprevir in HIV-HCV Coinfected Patients Who ...,"ANRS, Emerging Infectious Diseases",Janssen-Cilag Ltd.,COMPLETED,INTERVENTIONAL,False,2011-04-11,2011-04,2013-09,...,DERIVED,Telaprevir for HIV/hepatitis C virus-coinfecte...,Cotte L; Braun J; Lascoux-Combe C; Vincent C; ...,10.1093/cid/ciu659,[Journal Article],False,2014-08-18,True,bronze,False
3,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,...,RESULT,Apomorphine-induced blinking and yawning in he...,Blin O; Masson G; Azulay JP; Fondarai J; Serra...,10.1111/j.1365-2125.1990.tb03848.x,"[Clinical Trial, Journal Article, Randomized C...",False,1990-11-01,True,green,True
4,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,...,RESULT,Effect of acute and chronic lamotrigine on bas...,Ahmad S; Fowler LJ; Whitton PS,10.1038/sj.bjp.0705737,[Journal Article],False,2004-05-01,True,green,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NCT02481453,Rapamycine vs Placebo for the Treatment of Inc...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2015-06-25,2015-07-15,2018-01-22,...,BACKGROUND,Evaluation and construction of diagnostic crit...,Lloyd TE; Mammen AL; Amato AA; Weiss MD; Needh...,10.1212/WNL.0000000000000642,"[Journal Article, Review]",False,2014-07-29,True,green,True
496,NCT02051413,Beta-arrestins and Response to Venlafaxine in ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2014-01-31,2014-02-18,2018-05,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>
497,NCT02014883,Phase II Open Label Study Using Triheptanoin i...,Institut National de la Santé Et de la Recherc...,Ultragenyx Pharmaceutical Inc,COMPLETED,INTERVENTIONAL,False,2013-12-18,2013-12-04,2019-07-04,...,DERIVED,Triheptanoin dramatically reduces paroxysmal m...,Mochel F; Hainque E; Gras D; Adanyeguh IM; Cai...,10.1136/jnnp-2015-311475,"[Clinical Trial, Journal Article]",False,2015-11-03,True,hybrid,True
498,NCT02658253,Trial to Evaluate the Safety and Immunogenicit...,Institut National de la Santé Et de la Recherc...,"EVI Industries, Inc. | Recherche Clinique Pari...",COMPLETED,INTERVENTIONAL,False,2016-01-18,2016-01,2019-02-21,...,DERIVED,Progress and Insights Toward an Effective Plac...,Gamain B; Chêne A; Viebig NK; Tuikue Ndam N; N...,10.3389/fimmu.2021.634508,"[Journal Article, Review]",False,2021-02-25,True,gold,True


# OpenAlex :

#### Envoi de la requête :

In [82]:
# Query OpenAlex' API
select = [
    # 'id', 
    'doi', 
    # 'publication_date', 
    'type', 
    # 'open_access', 
    # 'authorships',
    # 'is_paratext',
    'primary_topic',    
]

# We return a list of dictionnaries, each dictionnary containing 
# publications_list = [{'doi': ..., 'primary_topic': ...}, {'doi': ..., 'primary_topic': ...}]
publications_list_opa = []

# Set of all the DOIs we need information for
dois = df_with_oa_status['doi'].dropna().unique()
num_dois = len(dois)

# For each DOI...
for i, doi in enumerate(dois):
    # Display the progress on a single line
    print(f'\r{i+1} / {num_dois}...', end='', flush=True)

    # Query OpenAlex's API
    query_url = f"https://api.openalex.org/works?filter=doi:{doi}&select={','.join(select)}&email={os.getenv('EMAIL_ADDRESS')}"
    response = session.get(query_url)
    response.raise_for_status()
    r_json = response.json()

    # Assert doi == strip_prefix(doi_openalex) ?
    
    if r_json['meta']['count'] == 1:
        result = response.json()['results'][0]

        # open_access = result['open_access']
        primary_topic = result['primary_topic']
        
        # Add the infos for the publication
        publications_list_opa.append(
            {
                'doi': doi,
                # 'publication_date': result['publication_date'], # similar to 'published_date' from Unpaywall 
                'type_openalex': result['type'],

                # Data from Unpaywall seems to be more up-to-date for now,
                # might be used in the futur in place of Unpaywall ?
                # 'is_oa_openalex': open_access['is_oa'],
                # 'oa_status': open_access['oa_status'],
                # 'repository': open_access['any_repository_has_fulltext'],

                # Add test ?
                # 'is_paratext': result['is_paratext'], # same as Unpaywall 
                
                'primary_topic': primary_topic['display_name'], 
                'subfield': primary_topic['subfield']['display_name'],
                'field':  primary_topic['field']['display_name'],
                'domain':  primary_topic['domain']['display_name'],
            }
        )
    
    else:
        # 0 or 2 results for a DOI
        print(f'error: {doi}, {r_json['meta']['count']} result')

387 / 387...

In [83]:
df_openalex = pd.DataFrame(publications_list_opa).astype(
    {
        'type_openalex': 'category', 
        'primary_topic': 'category',
        'subfield': 'category',
        'field': 'category',
        'domain': 'category',
    }
)
df_openalex

,doi,type_openalex,primary_topic,subfield,field,domain
0,10.1093/cid/ciz371,article,Human Immunodeficiency Virus/Acquired Immunode...,Virology,Immunology and Microbiology,Life Sciences
1,10.1093/cid/ciu659,article,Hepatitis C Infection and Treatment,Hepatology,Medicine,Health Sciences
2,10.1111/j.1365-2125.1990.tb03848.x,article,Olfactory Dysfunction in Health and Disease,Sensory Systems,Neuroscience,Life Sciences
3,10.1038/sj.bjp.0705737,article,Epilepsy and Seizures,Psychiatry and Mental health,Medicine,Health Sciences
4,10.1212/01.wnl.0000132840.40838.13,article,Epilepsy and Seizures,Psychiatry and Mental health,Medicine,Health Sciences
...,...,...,...,...,...,...
382,10.1016/j.nmd.2010.03.014,article,"Epidemiology, Classification, and Clinical Fea...",Epidemiology,Medicine,Health Sciences
383,10.1212/WNL.0000000000000642,article,"Epidemiology, Classification, and Clinical Fea...",Epidemiology,Medicine,Health Sciences
384,10.1136/jnnp-2015-311475,article,Effects of Ketogenic Diet on Health,Physiology,Medicine,Health Sciences
385,10.3389/fimmu.2021.634508,article,Malaria,"Public Health, Environmental and Occupational ...",Medicine,Health Sciences


#### Jointure des dataframes :

In [84]:
df_with_openalex = df_with_oa_status.merge(
    df_openalex, 
    on = ['doi'], 
    how = 'left',
    validate = 'many_to_one',
)
df_with_openalex

,NCTId,BriefTitle,LeadSponsorName,CollaboratorName,OverallStatus,StudyType,HasResults,StudyFirstPostDate,StartDate,CompletionDate,...,is_paratext,published_date,is_oa,oa_status,has_repository_copy,type_openalex,primary_topic,subfield,field,domain
0,NCT00658346,Molecular Diversity of HIV-1 Group O Strains a...,"ANRS, Emerging Infectious Diseases",<NA>,COMPLETED,OBSERVATIONAL,False,2008-04-15,2010-06,2016-09,...,False,2019-05-07,True,hybrid,False,article,Human Immunodeficiency Virus/Acquired Immunode...,Virology,Immunology and Microbiology,Life Sciences
1,NCT01647295,Social Interactions: Ocular Explorations and P...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,OBSERVATIONAL,False,2012-07-23,2013-02-25,2019-06-18,...,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
2,NCT01332955,Telaprevir in HIV-HCV Coinfected Patients Who ...,"ANRS, Emerging Infectious Diseases",Janssen-Cilag Ltd.,COMPLETED,INTERVENTIONAL,False,2011-04-11,2011-04,2013-09,...,False,2014-08-18,True,bronze,False,article,Hepatitis C Infection and Treatment,Hepatology,Medicine,Health Sciences
3,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,...,False,1990-11-01,True,green,True,article,Olfactory Dysfunction in Health and Disease,Sensory Systems,Neuroscience,Life Sciences
4,NCT01432821,Blinking and Yawning in Epilepsy: The Role of ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2011-09-13,2011-09,2014-12,...,False,2004-05-01,True,green,True,article,Epilepsy and Seizures,Psychiatry and Mental health,Medicine,Health Sciences
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NCT02481453,Rapamycine vs Placebo for the Treatment of Inc...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2015-06-25,2015-07-15,2018-01-22,...,False,2014-07-29,True,green,True,article,"Epidemiology, Classification, and Clinical Fea...",Epidemiology,Medicine,Health Sciences
496,NCT02051413,Beta-arrestins and Response to Venlafaxine in ...,Institut National de la Santé Et de la Recherc...,<NA>,COMPLETED,INTERVENTIONAL,False,2014-01-31,2014-02-18,2018-05,...,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
497,NCT02014883,Phase II Open Label Study Using Triheptanoin i...,Institut National de la Santé Et de la Recherc...,Ultragenyx Pharmaceutical Inc,COMPLETED,INTERVENTIONAL,False,2013-12-18,2013-12-04,2019-07-04,...,False,2015-11-03,True,hybrid,True,article,Effects of Ketogenic Diet on Health,Physiology,Medicine,Health Sciences
498,NCT02658253,Trial to Evaluate the Safety and Immunogenicit...,Institut National de la Santé Et de la Recherc...,"EVI Industries, Inc. | Recherche Clinique Pari...",COMPLETED,INTERVENTIONAL,False,2016-01-18,2016-01,2019-02-21,...,False,2021-02-25,True,gold,True,article,Malaria,"Public Health, Environmental and Occupational ...",Medicine,Health Sciences


In [85]:
df_with_openalex['type_openalex'].value_counts()

type_openalex
article     387
preprint      4
review        2
Name: count, dtype: int64

### Resultat final:

In [86]:
df_with_openalex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   NCTId                500 non-null    object  
 1   BriefTitle           500 non-null    string  
 2   LeadSponsorName      500 non-null    string  
 3   CollaboratorName     225 non-null    string  
 4   OverallStatus        500 non-null    category
 5   StudyType            500 non-null    category
 6   HasResults           500 non-null    boolean 
 7   StudyFirstPostDate   500 non-null    string  
 8   StartDate            500 non-null    string  
 9   CompletionDate       500 non-null    string  
 10  pmid                 394 non-null    object  
 11  type                 394 non-null    category
 12  title                394 non-null    object  
 13  authors              394 non-null    object  
 14  doi                  393 non-null    object  
 15  publication_types    39

# Export en CSV: 

In [87]:
df_with_openalex.to_csv(
    'Data/outputs/df_extract.csv',
    sep=';',
    index=False,
    encoding='utf-8-sig',
)